In [ ]:
import numpy as np
import os
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

directory_path = '/content/drive/MyDrive/new_speaker_identification/clips__test'

print(len(os.listdir(directory_path)))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
25


In [ ]:
import os
import numpy as np

def segment_npy_files(base_directory, segment_length, overlap=0):
    layers = [f"layer_{i}" for i in range(0, 25)]  # Erstellt eine Liste von Layer-Namen "layer_0" bis "layer_24"

    for speaker_idx in range(1, 26):  # Geht von Speaker1 bis Speaker25
        for layer_name in layers:
            source_directory = os.path.join(base_directory, f"Speaker{speaker_idx}", layer_name)
            target_directory = os.path.join(base_directory, f"Speaker{speaker_idx}", f"{layer_name}_no_overlap")

            if not os.path.exists(source_directory):
                print(f"Source directory does not exist: {source_directory}")
                continue  # Überspringt den aktuellen Layer, wenn das Quellverzeichnis nicht existiert

            # Überprüfung, ob das Zielverzeichnis bereits existiert und genügend Dateien enthält
            if os.path.exists(target_directory) and len(os.listdir(target_directory)) > 0:
                print(f"Target directory already processed: {target_directory}")
                continue  # Überspringt dieses Verzeichnis, wenn es bereits verarbeitet wurde

            os.makedirs(target_directory, exist_ok=True)
            print(f"Processing files in {source_directory}")

            for filename in os.listdir(source_directory):
                if filename.endswith(".npy"):
                    file_path = os.path.join(source_directory, filename)
                    data = np.load(file_path)
                    print(f"Loaded {filename} with data shape: {data.shape}")

                    # Segmentierung entlang der zweiten Dimension
                    step_size = segment_length - overlap
                    num_segments = (data.shape[1] - overlap) // step_size
                    print(f"Creating {num_segments} segments from {filename}")

                    segments = []
                    for i in range(num_segments):
                        if i * step_size + segment_length <= data.shape[1]:
                            segment = data[:, i * step_size: i * step_size + segment_length, :]
                            segments.append(segment)

                    for idx, segment in enumerate(segments):
                        segment_filename = f"{os.path.splitext(filename)[0]}_seg{idx}.npy"
                        segment_path = os.path.join(target_directory, segment_filename)
                        np.save(segment_path, segment)
                        print(f"Segment saved: {segment_path}")

# Beispiel für Nutzungsparameter
base_directory = '/content/drive/MyDrive/new_speaker_identification/clips__test'
segment_length = 100
overlap = 0

# Funktion aufrufen
segment_npy_files(base_directory, segment_length, overlap)